In [1]:
import numpy as np
import pandas as pd
import spacy
import nltk

In [2]:
df = pd.read_json("dataset//News_Category_Dataset_v3.json", lines=True)
print(df.head())

                                                link  \
0  https://www.huffpost.com/entry/covid-boosters-...   
1  https://www.huffpost.com/entry/american-airlin...   
2  https://www.huffpost.com/entry/funniest-tweets...   
3  https://www.huffpost.com/entry/funniest-parent...   
4  https://www.huffpost.com/entry/amy-cooper-lose...   

                                            headline   category  \
0  Over 4 Million Americans Roll Up Sleeves For O...  U.S. NEWS   
1  American Airlines Flyer Charged, Banned For Li...  U.S. NEWS   
2  23 Of The Funniest Tweets About Cats And Dogs ...     COMEDY   
3  The Funniest Tweets From Parents This Week (Se...  PARENTING   
4  Woman Who Called Cops On Black Bird-Watcher Lo...  U.S. NEWS   

                                   short_description               authors  \
0  Health experts said it is too early to predict...  Carla K. Johnson, AP   
1  He was subdued by passengers and crew when he ...        Mary Papenfuss   
2  "Until you have a dog y

In [3]:
df = df.drop(columns=["link", "date", "authors"])

In [4]:
df

,headline,category,short_description
0,Over 4 Million Americans Roll Up Sleeves For O...,U.S. NEWS,Health experts said it is too early to predict...
1,"American Airlines Flyer Charged, Banned For Li...",U.S. NEWS,He was subdued by passengers and crew when he ...
2,23 Of The Funniest Tweets About Cats And Dogs ...,COMEDY,"""Until you have a dog you don't understand wha..."
3,The Funniest Tweets From Parents This Week (Se...,PARENTING,"""Accidentally put grown-up toothpaste on my to..."
4,Woman Who Called Cops On Black Bird-Watcher Lo...,U.S. NEWS,Amy Cooper accused investment firm Franklin Te...
...,...,...,...
209522,RIM CEO Thorsten Heins' 'Significant' Plans Fo...,TECH,Verizon Wireless and AT&T are already promotin...
209523,Maria Sharapova Stunned By Victoria Azarenka I...,SPORTS,"Afterward, Azarenka, more effusive with the pr..."
209524,"Giants Over Patriots, Jets Over Colts Among M...",SPORTS,"Leading up to Super Bowl XLVI, the most talked..."
209525,Aldon Smith Arrested: 49ers Linebacker Busted ...,SPORTS,CORRECTION: An earlier version of this story i...


In [5]:
df.isna().sum()

headline             0
category             0
short_description    0
dtype: int64

In [6]:
print(df["headline"][0])
print(df.info())
print(df["category"].value_counts())
df["category"].count()

Over 4 Million Americans Roll Up Sleeves For Omicron-Targeted COVID Boosters
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 209527 entries, 0 to 209526
Data columns (total 3 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   headline           209527 non-null  object
 1   category           209527 non-null  object
 2   short_description  209527 non-null  object
dtypes: object(3)
memory usage: 4.8+ MB
None
category
POLITICS          35602
WELLNESS          17945
ENTERTAINMENT     17362
TRAVEL             9900
STYLE & BEAUTY     9814
PARENTING          8791
HEALTHY LIVING     6694
QUEER VOICES       6347
FOOD & DRINK       6340
BUSINESS           5992
COMEDY             5400
SPORTS             5077
BLACK VOICES       4583
HOME & LIVING      4320
PARENTS            3955
THE WORLDPOST      3664
WEDDINGS           3653
WOMEN              3572
CRIME              3562
IMPACT             3484
DIVORCE            3426
WORLD NEWS     

209527

In [7]:
df_limited = df.groupby('category').head(1000)


In [8]:
df_limited["category"].value_counts()
# df_limited["category"].count()

category
U.S. NEWS         1000
ARTS & CULTURE    1000
TRAVEL            1000
MONEY             1000
RELIGION          1000
LATINO VOICES     1000
IMPACT            1000
WEDDINGS          1000
COLLEGE           1000
PARENTS           1000
STYLE             1000
COMEDY            1000
GREEN             1000
TASTE             1000
HEALTHY LIVING    1000
THE WORLDPOST     1000
GOOD NEWS         1000
WORLDPOST         1000
FIFTY             1000
ARTS              1000
BLACK VOICES      1000
WOMEN             1000
HOME & LIVING     1000
QUEER VOICES      1000
PARENTING         1000
WORLD NEWS        1000
CULTURE & ARTS    1000
TECH              1000
SPORTS            1000
ENTERTAINMENT     1000
POLITICS          1000
WEIRD NEWS        1000
ENVIRONMENT       1000
EDUCATION         1000
CRIME             1000
SCIENCE           1000
WELLNESS          1000
BUSINESS          1000
STYLE & BEAUTY    1000
FOOD & DRINK      1000
MEDIA             1000
DIVORCE           1000
Name: count, dtype: int64

In [9]:
categories_to_remove = ["QUEER VOICES", "POLITICS", "IMPACT", "DIVORCE", "PARENTS", "WEIRD NEWS", "COLLEGE", \
        "LATINO VOICES", "CULTURE & ARTS", "BLACK VOICES", "GREEN", "TASTE", "WEDDINGS", "U.S. NEWS", "FIFTY", "ARTS" \
            "WOMEN",'SCIENCE', 'WELLNESS', 'BUSINESS' \
, 'STYLE & BEAUTY', 'FOOD & DRINK', 'MEDIA', 'HOME & LIVING', 'WOMEN', \
 'MONEY', 'RELIGION', 'ARTS & CULTURE', 'STYLE', 'HEALTHY LIVING', \
 'THE WORLDPOST', 'GOOD NEWS', 'WORLDPOST', 'ARTS' \
    ,'COMEDY', 'PARENTING', 'WORLD NEWS', "ENTERTAINMENT", \
 'ENVIRONMENT', 'CRIME', 'HEALTHY LIVING' \
    ]

df_limited = df_limited[~df_limited['category'].isin(categories_to_remove)]
df_limited["category"].value_counts()

unique_categories = df_limited['category'].unique()
print(unique_categories)

['TECH' 'SPORTS' 'EDUCATION' 'TRAVEL']


In [10]:
nlp = spacy.load('en_core_web_lg')


def cleaning_text(text):

    # print(text)
    text = text.lower()
    doc = nlp(text)

    new_line = []

    for w in doc :
        if(w.is_stop == False and w.is_punct == False and w.is_space == False):
            new_line.append(w.lemma_)
    # new_line = " ".join(new_line)

    # next = []
    # print(new_line)

    # for word in new_line :
    #     next.append(stemmer.stem(word))
    
    new_line = " ".join(new_line)

    return new_line

In [11]:
text = cleaning_text(df_limited["short_description"][13])
print(text)

man claim scamme people platform cause popular streamer consider twitch boycott


In [12]:
df_limited["short_description"] = df_limited['short_description'].apply(cleaning_text)

In [13]:
print(df_limited.head())

                                             headline category  \
13  Twitch Bans Gambling Sites After Streamer Scam...     TECH   
17  Maury Wills, Base-Stealing Shortstop For Dodge...   SPORTS   
26  Las Vegas Aces Win First WNBA Title, Chelsea G...   SPORTS   
55  TikTok Search Results Riddled With Misinformat...     TECH   
61  Boston Marathon To Make Race More Inclusive Fo...   SPORTS   

                                    short_description  
13  man claim scamme people platform cause popular...  
17  maury will help los angeles dodgers win world ...  
26       las vegas professional sport champion sunday  
55  u.s firm monitor false online claim report sea...  
61  race organizer nonbinary athlete will register...  


In [14]:
df_limited["short_description"][13]

'man claim scamme people platform cause popular streamer consider twitch boycott'

In [15]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional, Dropout


In [16]:
model = Sequential()

print(df_limited["short_description"])


13        man claim scamme people platform cause popular...
17        maury will help los angeles dodgers win world ...
26             las vegas professional sport champion sunday
55        u.s firm monitor false online claim report sea...
61        race organizer nonbinary athlete will register...
                                ...                        
132603    education reformer union leader pretty pay att...
132638    skin color race matter historical discriminati...
132642                                                     
132704    admission officer value test well align common...
132778    fascinating take e color school speed uptake c...
Name: short_description, Length: 4000, dtype: object


In [24]:
tokenizer = Tokenizer(num_words=10000)

tokenizer.fit_on_texts(df_limited["short_description"])

description = tokenizer.texts_to_sequences(df_limited["short_description"])

maxi = 0
length = len(description)


for i in range(0, length):
    if len(description[i]) > maxi: 
        maxi = len(description[i])

print(maxi)


38


In [25]:
description

[[80, 283, 3918, 13, 487, 369, 454, 3919, 226, 3920, 3921],
 [3922, 262, 40, 263, 305, 1637, 47, 11, 284, 597, 204, 1154, 2635, 423],
 [2031, 1638, 322, 147, 529, 598],
 [29,
  27,
  1155,
  1639,
  2032,
  227,
  283,
  137,
  424,
  205,
  1356,
  99,
  1357,
  2636,
  425,
  108,
  306,
  3923,
  2637],
 [347, 1358, 3924, 138, 262, 3925, 80, 51, 2033, 244, 3926, 6, 1020, 530],
 [3927, 1640, 1641, 1156, 41, 206, 1157, 264, 228],
 [2638,
  3928,
  599,
  3929,
  3930,
  29,
  27,
  132,
  531,
  814,
  488,
  911,
  390,
  815,
  100,
  80,
  60,
  3931],
 [1359, 52, 600, 426, 601, 3932, 3933, 3934, 2639, 602, 532],
 [178, 229, 489, 29, 27, 1, 17, 1642, 533, 137, 5],
 [2034, 3935, 2035, 3936, 52, 42, 603, 1158, 1640, 3937, 162, 816, 604, 2640],
 [3938, 285, 3939, 2641, 1159, 1643, 53, 912, 427],
 [2036,
  8,
  1644,
  817,
  14,
  534,
  92,
  5,
  428,
  2642,
  1645,
  7,
  1360,
  101,
  2037,
  8,
  535,
  657,
  3940,
  204,
  2643],
 [3941, 3942, 2038, 139, 536, 14, 391, 818, 10

In [26]:
description = pad_sequences(description, padding='post', maxlen=maxi)

In [38]:
description.shape

(4000, 38)

In [48]:
from tensorflow.keras.layers import Flatten

model = Sequential()
model.add(Embedding(10000, 64, input_length=maxi))
model.add(LSTM(64, return_sequences=True))
model.add(LSTM(64))
model.add(Flatten())  # Flatten the output to 2D
model.add(Dense(5, activation='softmax'))  # Adjust the number of classes
model.compile(optimizer='adam', loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True) , metrics=['accuracy'])

In [49]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df_limited["category"] = le.fit_transform(df_limited["category"])


In [51]:
model.fit(description, df_limited['category'], epochs=10)

Epoch 1/10


ValueError: Attr 'Toutput_types' of 'OptionalFromValue' Op passed list of length 0 less than minimum 1.